In [3]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite
from cProfile import label
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

In [4]:

df = pd.read_csv('../data/taxon-exporter.csv', header=0)
G = nx.Graph()  # or DiGraph, MultiGraph, MultiDiGraph, etc

taxons = {}
purpose = {}
       
        
for index, row in df.iterrows():
    if(row['Taxon'] not in taxons):
        purpose = {}
    
    if(row['Purpose'] == "B"):
        purposeCode = "Breeding"
    if(row['Purpose'] == "E"):
        purposeCode = "Educational"
    if(row['Purpose'] == "G"):
        purposeCode = "Garden"
    if(row['Purpose'] == "H"):
        purposeCode = "Hunting"
    if(row['Purpose'] == "L"):
        purposeCode = "Law"
    if(row['Purpose'] == "M"):
        purposeCode = "Medical"
    if(row['Purpose'] == "R"):
        purposeCode = "Reintroduction to wild"
    if(row['Purpose'] == "P"):
        purposeCode = "Personal"
    if(row['Purpose'] == "Q"):
        purposeCode = "Circus"
    if(row['Purpose'] == "S"):
        purposeCode = "Scientific"
    if(row['Purpose'] == "T"):
        purposeCode = "Commercial"
    if(row['Purpose'] == "Z"):
        purposeCode = "Zoo"
    if(row['Purpose'] == "Unspecified"):
        purposeCode = "Unspecified"
        
    purposeDetails = {purposeCode: "X"}
        
    purpose.update(purposeDetails)
    attributes = {"Purpose": purpose}

    taxons.update({row['Taxon']: row['Taxon']})
    G.add_node(row['Exporter'], **attributes)


for index, row in df.iterrows():
    for index, compareRow in df.iterrows():
        if row['Exporter'] != compareRow['Exporter'] and row['Taxon'] == compareRow['Taxon']:
            G.add_edge(row['Exporter'], compareRow['Exporter'], country = row['Taxon'])
    print(index)
nx.write_gml(G, "exporter-exporter-taxon.gml")



27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378
27378


KeyboardInterrupt: 